### Imports

In [ ]:
## pip install keras
## pip install tensorflow
## py -m venv keras
## pip show tensorflow

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [3]:
spam = pd.read_csv("spam.csv")
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
## Preprocessamento (tratamento)
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam["Category"])
print(y)

[0 0 1 ... 0 0 0]


In [7]:
## Dividir Dados em Treino
mensagens = spam["Message"].values
X_train,X_test, y_train,y_test = train_test_split(mensagens,y,test_size = 0.3)
print(X_train)

['Moji i love you more than words. Have a rich day' 'Okie...'
 "And also I've sorta blown him off a couple times recently so id rather not text him out of the blue looking for weed"
 ... 'Ü say until like dat i dun buy ericsson oso cannot oredi lar...'
 'Do u noe how 2 send files between 2 computers?'
 'Email AlertFrom: Jeri StewartSize: 2KBSubject: Low-cost prescripiton drvgsTo listen to email call 123']


In [8]:
vetorizador = CountVectorizer()
vetorizador.fit(X_train)
X_train = vetorizador.transform(X_train)
X_test = vetorizador.transform(X_test)
print(X_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
## Criando Modelo
modelo = Sequential()
modelo.add(Dense( units=10, activation="relu", input_dim= X_train.shape[1] ))
modelo.add(Dropout(0.1))
modelo.add(Dense( units=8, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense( units=1, activation="sigmoid"))

In [15]:
## Montar a estrutura do Modelo
modelo.compile(loss = "mean_squared_error", optimizer = "adam", metrics= ["accuracy"] )
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                71370     
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 71,467
Trainable params: 71,467
Non-trainable params: 0
_________________________________________________________________


In [16]:
## Treinar o Modelo
modelo.fit(X_train, y_train,epochs=20,batch_size=10, verbose= True, validation_data=(X_test,y_test))

Epoch 1/20


C:\Users\thiago.ao\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


390/390 [==============================] - 1s 2ms/step - loss: 0.0844 - accuracy: 0.9292 - val_loss: 0.0191 - val_accuracy: 0.9809
Epoch 2/20
390/390 [==============================] - 1s 1ms/step - loss: 0.0118 - accuracy: 0.9890 - val_loss: 0.0140 - val_accuracy: 0.9827
Epoch 3/20
390/390 [==============================] - 1s 2ms/step - loss: 0.0061 - accuracy: 0.9949 - val_loss: 0.0124 - val_accuracy: 0.9839
Epoch 4/20
390/390 [==============================] - 1s 2ms/step - loss: 0.0034 - accuracy: 0.9967 - val_loss: 0.0113 - val_accuracy: 0.9862
Epoch 5/20
390/390 [==============================] - 1s 2ms/step - loss: 0.0027 - accuracy: 0.9969 - val_loss: 0.0125 - val_accuracy: 0.9850
Epoch 6/20
390/390 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 0.9985 - val_loss: 0.0118 - val_accuracy: 0.9850
Epoch 7/20
390/390 [==============================] - 1s 2ms/step - loss: 0.0015 - accuracy: 0.9987 - val_loss: 0.0102 - val_accuracy: 0.9892
Epoch 8/20
390/39

In [18]:
loss,accuracy = modelo.evaluate(X_test,y_test)
print("Loss:",loss)
print("Accuracy:",accuracy)

53/53 [==============================] - 0s 1ms/step - loss: 0.0134 - accuracy: 0.9827
Loss: 0.013409281149506569
Accuracy: 0.9826555252075195


In [19]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

53/53 [==============================] - 0s 1ms/step
[[4.1959748e-08]
 [3.1987443e-06]
 [9.9999654e-01]
 ...
 [2.5224725e-03]
 [2.9204267e-07]
 [1.2540960e-04]]


In [21]:
prev = (nova_previsao > 0.5)
print(prev)

[[False]
 [False]
 [ True]
 ...
 [False]
 [False]
 [False]]


In [22]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1432    3]
 [  26  211]]
